In [1]:
from SNAKE_AI import SnakeGame
import tensorflow as tf
from keras import Sequential, layers, optimizers
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import random 

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from numpy import float32


def create_model(input_shape, output):
    model = Sequential()
    model.add(layers.Dense(12, input_shape=(input_shape[1],), activation="relu", kernel_regularizer=tf.initializers.random_uniform()))
    model.add(layers.Dense(12, activation="relu", kernel_regularizer=tf.initializers.he_uniform()))
    model.add(layers.Dense(4, activation="relu", kernel_regularizer=tf.initializers.he_uniform()))
    model.add(layers.Dense(output, activation="linear", kernel_regularizer=tf.initializers.he_uniform()))
    model.compile(optimizer=optimizers.adam_v2.Adam(0.01), loss="mse")
    return model

def state_helper(state):
    direction, (sx, sy),(fx,fy)=state
    return np.array([[direction,sx,sy,fx,fy]] ,dtype=np.float32)

def train(minibatch):
    x_stack = np.empty(0).reshape(1,5)
    y_stack = np.empty(0).reshape(1,3)

    for state, action, next_state, reward, done in minibatch:
        predict = model.predict(state)
        if done:
            predict[0,action] = reward
        else:
            predict[0,action] = reward+ gamma*np.max(target.predict(next_state))

        np.vstack([x_stack, state])
        np.vstack([y_stack, predict[0]])
    model.fit(x_stack, y_stack, batch_size=32, verbose=0)
    



In [5]:
epi = 10000
replay_memory = deque(maxlen=50000)
model = create_model((None,5), 3)
target = create_model((None,5), 3)
gamma = .9
score_list = []
env = SnakeGame()

#e greedy
e = 1.0
e_decay = .995
e_min = .01

for i in range(epi):
    game_over = False
    score = 0
    state = env.reset()
    # print(state)
    while not game_over:
        state = state_helper(state)
        predict = model.predict(state)
        if np.random.rand() < e:
            action = np.argmax(np.random.rand(3))
        else:
            action = np.argmax(predict[0])

        next_state, reward, done = env.step(action)
        replay_memory.append((state, action, state_helper(next_state), reward, done))
        score += reward
        if done:
            score_list.append(score)
            break
        state = next_state
    if e>e_min:
        e*=e_decay

    if len(replay_memory)>32:
        minibatch = random.sample(replay_memory,32)
    print(f"epi:{i}, score:{score} e:{e}")
    if i%10==0:
        target.set_weights(model.get_weights())



epi:0, score:-10 e:0.995
epi:1, score:-10 e:0.990025
epi:2, score:-10 e:0.985074875
epi:3, score:-10 e:0.9801495006250001
epi:4, score:0 e:0.9752487531218751
epi:5, score:-10 e:0.9703725093562657
epi:6, score:-10 e:0.9655206468094844
epi:7, score:-10 e:0.960693043575437
epi:8, score:-10 e:0.9558895783575597
epi:9, score:-10 e:0.9511101304657719
epi:10, score:-10 e:0.946354579813443
epi:11, score:0 e:0.9416228069143757
epi:12, score:-10 e:0.9369146928798039
epi:13, score:0 e:0.9322301194154049
epi:14, score:-10 e:0.9275689688183278
epi:15, score:0 e:0.9229311239742362
epi:16, score:-10 e:0.918316468354365
epi:17, score:-10 e:0.9137248860125932
epi:18, score:-10 e:0.9091562615825302
epi:19, score:-10 e:0.9046104802746175
epi:20, score:-10 e:0.9000874278732445
epi:21, score:-10 e:0.8955869907338783
epi:22, score:-10 e:0.8911090557802088
epi:23, score:-10 e:0.8866535105013078
epi:24, score:-10 e:0.8822202429488013
epi:25, score:-10 e:0.8778091417340573
epi:26, score:-10 e:0.873420096025387

KeyboardInterrupt: 

: 